In [5]:
from PIL import Image
import requests
import torch
from torchvision import io
from typing import Dict
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, TextIteratorStreamer
import sys
from threading import Thread


from qwen_vl_utils import process_vision_info

In [6]:
from tool_server.tf_eval.utils.utils import *

In [2]:
# Load the model in half-precision on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "/mnt/petrelfs/songmingyang/songmingyang/model/tool-augment/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("/mnt/petrelfs/songmingyang/songmingyang/model/tool-augment/Qwen2-VL-7B-Instruct")


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [00:26<00:00,  5.34s/it]


In [9]:
# Image
url = "/mnt/petrelfs/songmingyang/code/tools/test_imgs/roxy.jpeg"
image = Image.open(url)
image_base64=pil_to_base64(image)
image_base64 = f"data:image/jpeg;base64,{image_base64}"
conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]
image_inputs, video_inputs = process_vision_info(conversation)

In [3]:


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt], images=[image], padding=True, return_tensors="pt"
)
inputs = inputs.to("cuda")
# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
print(output_text)


["The image is an illustration of an anime-style character. The character has long, flowing blue hair and is wearing a beige coat with a high collar and a black dress underneath. The coat has a brownish hue and features a large, decorative button on the front. The character is holding a large, mechanical-looking sword with a black and silver design. The sword has a long, cylindrical handle and a sharp blade. In the background, there is a floating blue crystal or gemstone, and the character appears to be in a snowy or cold environment, as suggested by the white, snowy landscape and the character's attire. The overall style is"]


In [12]:
tokenizer = processor.tokenizer
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=15)



In [14]:
# Image
url = "/mnt/petrelfs/songmingyang/code/tools/test_imgs/roxy.jpeg"
image = Image.open(url)

conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt], images=[image], padding=True, return_tensors="pt"
)
inputs = inputs.to("cuda")
kwargs = {
            **inputs,  # 解包原有的 inputs 字典
            "streamer":streamer,
            'max_new_tokens':1024 # 添加额外的参数
    
        }
thread = Thread(target=model.generate, kwargs=kwargs,)
thread.start()

for new_text in streamer:
    print(new_text, end='', flush=True)

The image is an illustration of an anime-style character. The character has long, flowing blue hair and is wearing a beige coat with a high collar and a black dress underneath. The coat has a brownish hue and features a large, decorative button on the front. The character is holding a large, mechanical-looking sword with a black and silver design. The sword has a long, cylindrical handle and a sharp blade. In the background, there is a floating blue crystal or gemstone, and the character appears to be in a snowy or cold environment, as suggested by the white, snowy landscape and the character's attire. The overall style is vibrant and detailed, with a focus on the character's expression and the mechanical elements of the sword.